In [1]:
import streamlit as st
import gspread as gs 

SPREADSHEET = 'portfolio'

gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
ss_portfolio = gc.open_by_key(st.secrets['portfolio'].spreadsheet_key).worksheet(st.secrets['portfolio'].worksheet_name)

In [2]:
import yfinance as yf
import pandas as pd

eurusd: pd.DataFrame = yf.download('EURUSD=X', start='2021-01-01')[['Close']].reset_index()

[*********************100%***********************]  1 of 1 completed


In [48]:
df_portfolio = pd.DataFrame(ss_portfolio.get_all_records())
df_portfolio['EURUSD'] = df_portfolio.apply(lambda row: eurusd.query("Date == @row['Date']")['Close'].values[0], axis=1)
df_portfolio['Total€'] = df_portfolio.apply(lambda row: row['Total'] / (1 if row['Currency'] == "€" else row['EURUSD']), axis=1)
df_portfolio.head()


,Date,Portfolio,Account,Asset,Price,Amount,Fees,Total,Total€,Currency,EURUSD,Out,Profit
0,2021-04-30,ZEN REMIX,eToro,Deposit $,,,,598.0,493.110802,$,1.212709,,
1,2021-04-30,ZEN REMIX,eToro,GREENBULL,598,1,,598.0,493.110802,$,1.212709,,
2,2021-07-30,ZEN REMIX,eToro,Deposit $,,,,585.0,491.886106,$,1.189300,,
3,2021-07-30,ZEN REMIX,eToro,GREENBULL,585,1,,585.0,491.886106,$,1.189300,,
4,2021-08-10,DMA US PEA,B.Direct PEA,Deposit €,,,,100.0,100.000000,€,1.173847,,


In [49]:
import numpy as np

df_deposit = df_portfolio[(df_portfolio['Portfolio'] == "ZEN REMIX") & (df_portfolio['Asset'].str.contains("Deposit"))]
df_deposit['Deposit€'] = [np.sum(df_deposit.loc[:idx]['Total€']) for idx in df_deposit.index]

df_invested = df_portfolio[(df_portfolio['Portfolio'] == "ZEN REMIX") & (~df_portfolio['Asset'].str.contains("Deposit"))]
df_invested['Invested€'] = [np.sum(df_invested.loc[:idx]['Total€']) for idx in df_invested.index]


C:\Users\vcordier\AppData\Local\Temp\ipykernel_22004\2892415915.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\vcordier\AppData\Local\Temp\ipykernel_22004\2892415915.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [54]:
df = pd.concat([df_deposit, df_invested])[['Date', 'Deposit€', 'Invested€']].sort_index()
pd.date_range(start=min(df_invested['Date']),end=max(df_invested['Date']))

DatetimeIndex(['2021-04-30', '2021-05-01', '2021-05-02', '2021-05-03',
               '2021-05-04', '2021-05-05', '2021-05-06', '2021-05-07',
               '2021-05-08', '2021-05-09',
               ...
               '2022-10-03', '2022-10-04', '2022-10-05', '2022-10-06',
               '2022-10-07', '2022-10-08', '2022-10-09', '2022-10-10',
               '2022-10-11', '2022-10-12'],
              dtype='datetime64[ns]', length=531, freq='D')

In [41]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(x=df['Date'], y=df['Deposit€']))
fig.add_trace(go.Bar(x=df['Date'], y=df['Invested€']))
fig.show()

In [16]:
import plotly.express as px

fig = px.line(df_invested, x='Date', y='Value€')
fig = px.line(df_deposit, x='Date', y='Value€')
fig.show()